# Hate Speech Detector - EN - Features extraction for Conv. & LSTM model

Based on [this notebook](https://github.com/t-davidson/hate-speech-and-offensive-language/blob/master/classifier/final_classifier.ipynb).

In [1]:
import os
import pandas as pd
import csv
import numpy as np
import pickle
from klepto.archives import dir_archive
import sys
import nltk
import string
import re
import fasttext
from pymagnitude import *

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer

from tqdm.notebook import tqdm
%matplotlib inline

In [2]:
MODEL = 'conv'

## Davidson et al. data loading

### Classes pre:
    0 - hate speech
    1 - offensive language
    2 - neither

In [3]:
if not os.path.exists('hsd/DavidsonEtAl/perfect_data.pkl'):
    tweets, labels = [], []
    with open('hsd/DavidsonEtAl/labeled_data.csv', 'r') as f:
        for d in tqdm(list(csv.reader(f))[1:]):
            tweets.append(d[6])  # tweet
            labels.append(d[5])  # class
    with open('hsd/DavidsonEtAl/perfect_data.pkl', 'w') as f:
        def chcl(c):
            return 0 if c=='2' else 1
        labels = list(map(chcl, labels))
        pickle.dump((tweets, labels), f)
else:
    with open('hsd/DavidsonEtAl/perfect_data.pkl', 'rb') as f:
        tweets, labels = pickle.load(f)

### Classes post:
    0 - no hate
    1 - hate speech

In [4]:
print('Tweets: {}'.format(len(tweets)))
print('Labels: {}'.format(len(labels)))

Tweets: 24783
Labels: 24783


In [5]:
list(zip(tweets[:5], labels[:5]))

[("!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...",
  0),
 ('!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!',
  1),
 ('!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit',
  1),
 ('!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny', 1),
 ('!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;',
  1)]

## Features extraction

In [6]:
def word_tokenization(tweet):
    lemmatizer = WordNetLemmatizer() 
    tokens = word_tokenize(tweet)
    words = [word for word in tokens if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    words = [lemmatizer.lemmatize(w) for w in words]
    return words

def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    #hashtag_regex = '#[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    #parsed_text = re.sub(hashtag_regex, '', parsed_text)
    list_words = word_tokenization(parsed_text)
    parsed_text = " ".join(list_words)
    return parsed_text

def basic_tokenize(tweet):
    tweet = " ".join(re.split(" ", tweet.lower())).strip()
    return tweet.split()

def get_pos_string(tweet):
    text = preprocess(tweet)
    tokens = basic_tokenize(text)
    tags = nltk.pos_tag(tokens)
    tag_list = [x[1] for x in tags]
    tag_str = ' '.join(tag_list)
    
    return tag_str

def pad_words(words, length):
    if len(words) >= length:
        return words[:length]
    else:
        additional = length - len(words)
        return words + ['EMPTY']*additional

### Median sentences length

In [7]:
def median_sentences_length(data):
    all_lengths, wt_lengths, pos_lengths = [], [], []
    for d in data:
        sentence = preprocess(d)
        pos_string = get_pos_string(d)
        all_lengths.append(len(sentence.split(' ')))
        all_lengths.append(len(pos_string.split(' ')))
        wt_lengths.append(len(sentence.split(' ')))
        pos_lengths.append(len(pos_string.split(' ')))
    
    return int(np.median(all_lengths)), int(np.median(wt_lengths)), int(np.median(pos_lengths))

In [8]:
opt_length, opt_wt_length, opt_pos_length = median_sentences_length(tweets)
dim = 6*20 if MODEL == 'conv' else 200

print('Optimal all length: {}'.format(opt_length))
print('Optimal sentence length: {}'.format(opt_wt_length))
print('Optimal pos sentence length: {}'.format(opt_pos_length))

Optimal all length: 8
Optimal sentence length: 8
Optimal pos sentence length: 8


### Supervised fastText wordtokens training

In [9]:
if not os.path.exists('hsd/DavidsonEtAl/fasttext.ft'):
    with open('hsd/DavidsonEtAl/fasttext.ft', 'a') as f:
        for t, l in list(zip(tweets, labels)):
            f.write('__label__{} {}\n'.format(l, preprocess(t)))

# load fasttext model or train & save if none
if os.path.exists('hsd/DavidsonEtAl/fasttext_{}.bin'.format(MODEL)):
    ft_model = fasttext.load_model('hsd/DavidsonEtAl/fasttext_{}.bin'.format(MODEL))
else:
    ft_model = fasttext.train_supervised('hsd/DavidsonEtAl/fasttext.ft',
                                         lr=0.5, epoch=50, wordNgrams=3, dim=dim)
    ft_model.save_model('hsd/DavidsonEtAl/fasttext_{}.bin'.format(MODEL))

### Wordtoken features

In [10]:
def get_wordtoken_fts(data, length):
    
    sentences_words = []
    for d in data:
        sentence = preprocess(d)
        sentences_words.append(sentence.split(' '))
    
    sentences_words = [pad_words(sw, length) for sw in sentences_words]
    vecs = Magnitude(MagnitudeUtils.download_model('http://magnitude.plasticity.ai/fasttext/light/crawl-300d-2M.magnitude'))
    
    ft_matrices = []
    for sw in sentences_words:
        ft_matrix = []
        for w in sw:
            ft_matrix.append(ft_model[w])
        ft_matrices.append(ft_matrix)
    
    return ft_matrices

In [11]:
wordtoken_features = get_wordtoken_fts(tweets, opt_wt_length)

In [12]:
wordtoken_features[0]

[array([ 5.41524410e-01, -1.54188037e-01,  2.22213462e-01, -1.86897978e-01,
         6.22597076e-02,  2.36525789e-01, -7.90781975e-02, -1.98556215e-01,
        -4.55635339e-02, -1.12501532e-01,  1.43155068e-01, -6.11903183e-02,
        -3.29692155e-01,  1.50330774e-02,  1.24902926e-01, -1.31155932e-02,
        -1.71159636e-02,  3.32070701e-02,  6.12771735e-02,  1.63981289e-01,
        -3.31126116e-02, -1.87257841e-01, -5.91149628e-02,  4.78049032e-02,
        -1.51351154e-01,  4.95405868e-02, -2.91528925e-02,  5.43664731e-02,
         1.77114949e-01, -1.78933576e-01,  1.48397222e-01,  1.19729511e-01,
         3.68238278e-02, -2.49445178e-02,  1.30025610e-01, -1.13121979e-02,
        -1.73780061e-02,  1.00510366e-01, -2.74626408e-02,  1.14193611e-01,
        -2.05562115e-01, -1.22029886e-01, -1.32308647e-01,  2.74365675e-02,
         3.30604352e-02,  1.61028758e-03,  4.98550385e-02, -4.27967682e-03,
         1.02420539e-01,  7.56832212e-02,  6.71983212e-02, -1.36099532e-01,
         2.9

### Supervised fastText pos training

In [13]:
if not os.path.exists('hsd/DavidsonEtAl/fasttext_pos.ft'):
    with open('hsd/DavidsonEtAl/fasttext_pos.ft', 'a') as f:
        for t, l in list(zip(tweets, labels)):
            f.write('__label__{} {}\n'.format(l, get_pos_string(t)))

# load fasttext pos model or train & save if none
if os.path.exists('hsd/DavidsonEtAl/fasttext_pos_{}.bin'.format(MODEL)):
    ft_pos_model = fasttext.load_model('hsd/DavidsonEtAl/fasttext_pos_{}.bin'.format(MODEL))
else:
    ft_pos_model = fasttext.train_supervised('hsd/DavidsonEtAl/fasttext_pos.ft',
                                             lr=0.5, epoch=50, wordNgrams=3, dim=dim)
    ft_pos_model.save_model('hsd/DavidsonEtAl/fasttext_pos_{}.bin'.format(MODEL))

### Part of speech (PoS) features

In [15]:
pos_features = get_pos_fts(tweets, opt_pos_length)

In [16]:
pos_features[0]

[array([-2.46898711e-01,  5.58380902e-01, -7.11930037e-01, -9.59126651e-02,
         1.82142839e-01,  5.09144850e-02,  2.31609084e-02, -1.64363328e-02,
         4.00507241e-01,  2.78016236e-02, -2.04522535e-02,  3.71972993e-02,
         1.77888483e-01, -9.19842422e-02, -1.58033683e-04, -1.00522779e-03,
         2.50629857e-02,  7.61646591e-03, -3.22217168e-03, -3.02780736e-02,
        -1.32991940e-01,  8.56668279e-02,  8.51784945e-02, -1.90940909e-02,
         2.81706117e-02,  6.70357561e-03,  7.36798272e-02, -2.61438359e-02,
         2.63021197e-02,  1.21873833e-01, -1.89289570e-01,  8.87137931e-03,
        -1.27736673e-01, -4.48044062e-01, -5.55836141e-01,  6.86058588e-03,
         4.86441217e-02, -1.95673376e-01,  2.29656305e-02, -2.70860363e-02,
         1.24467187e-01,  6.79230615e-02,  1.91673413e-01, -2.21526623e-02,
         7.93640539e-02, -7.77081475e-02, -9.78923663e-02, -2.34863579e-01,
        -3.66918743e-01, -2.57314891e-01, -1.05987035e-01,  1.71188131e-01,
        -1.2

In [17]:
np.array(wordtoken_features).shape

(24783, 8, 120)

In [18]:
np.array(pos_features).shape

(24783, 8, 120)

### All features

In [19]:
#Now join them all up
features = np.concatenate([wordtoken_features, pos_features], axis=1)

In [20]:
features.shape

(24783, 16, 120)

In [21]:
features[0]


array([[ 0.5415244 , -0.15418804,  0.22221346, ..., -0.05535417,
         0.02035121, -0.03372673],
       [ 0.08779506, -0.03319542,  0.05781299, ..., -0.04436125,
         0.01935646, -0.02882041],
       [-0.35537452,  0.16440548, -0.22475338, ...,  0.21329857,
        -0.08368158,  0.14254798],
       ...,
       [-0.24689871,  0.5583809 , -0.71193004, ..., -0.1025828 ,
         0.04111252,  0.00291486],
       [-0.24689871,  0.5583809 , -0.71193004, ..., -0.1025828 ,
         0.04111252,  0.00291486],
       [-0.24689871,  0.5583809 , -0.71193004, ..., -0.1025828 ,
         0.04111252,  0.00291486]], dtype=float32)

## Save features & labels

In [22]:
archive = dir_archive('hsd/DavidsonEtAl/X_y_{}'.format(MODEL), {'features': features, 'labels': labels,
                                                                'wt_num': np.array(wordtoken_features).shape[1]}, serialized=True)
archive.dump()
del archive